# 1) Create source sound collection

This notebook includes the code to create the collection of sounds that will later be used as source material for our audio mosaicing application. The collection of sounds is created by defining a number of queries to be performed using the Freesound API and concatenanting the results of each query. A number of metadata fields are stored for each sound in the collection and saved into a Pandas DataFrame object and CSV file in disk. For each sound in the collection, we also download an OGG preview and store it in disk.

This notebook uses the `freesound` Python package for interacting with the Freesound API. The source code for this package can be found here: https://github.com/mtg/freesound-python. In this repository you'll find a Python script with [examples](https://github.com/MTG/freesound-python/blob/master/examples.py) to learn how to interact with the API. Nevertheless, if you are further interested in the Freesound API, check the [API documentation](http://freesound.org/docs/api/) which provides more information.

**NOTE**: A Freesound API key is provided in this notebook, but you should make a Freesound account and get your own key. You can get a key here: https://freesound.org/apiv2/apply/

In [ ]:
# Essentia
!pip install essentia
# Freesound-python
!pip install git+https://github.com/mtg/freesound-python.git
# Mount drive and cd to notebook folder
from google.colab import drive
drive.mount('/content/drive')


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 61.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/mtg/freesound-python.git to /tmp/pip-req-build-zk9ifthj
  Running command git clone --filter=blob:none --quiet https://github.com/mtg/freesound-python.git /tmp/pip-req-build-zk9ifthj
  Resolved https://github.com/mtg/freesound-python.git to commit 14f2e6d35587181fb376f36fb2cfab4941c05b15
  Preparing metadata (setup.py) ... done
  Created wheel for freesound-python: filename=freesound_python-1.1-py3-none-any.whl size=6443 sha256=6e446817101237879027e7a1bcdfdfd41c179e7fb0fa97ae6362cb1c97db2606
  Stored in directory: /tmp/pip-ephem-wheel-cache-knry203e/wheels/0e/40/c2/8769649bfe986e312e67c9fd16cfcca06fd386442f6f6ef45b
Successfully built freesound-python
Mounted at /content/drive


In [7]:
%cd '/content/drive/MyDrive/Term2/ASMLab/Freesound/Self work'

/content/drive/MyDrive/Term2/ASMLab/Freesound/Self work


In [12]:
import os
import pandas as pd
import numpy as np
import freesound
from IPython.display import display

FREESOUND_API_KEY = ''  # Please replace by your own Freesound API key
FILES_DIR = 'files'  # Place where to store the downloaded diles. Will be relative to the current folder.
DATAFRAME_FILENAME = 'dataframe.csv'  # File where we'll store the metadata of our sounds collection
FREESOUND_STORE_METADATA_FIELDS = ['id', 'name', 'username', 'previews', 'license', 'tags']  # Freesound metadata properties to store

freesound_client = freesound.FreesoundClient()
freesound_client.set_token(FREESOUND_API_KEY)
if not os.path.exists(FILES_DIR): os.mkdir(FILES_DIR)

In [13]:
# Define some util functions

def query_freesound(query, filter, num_results=10):
    """Queries freesound with the given query and filter values.
    If no filter is given, a default filter is added to only get sounds shorter than 30 seconds.
    """
    if filter is None:
        filter = 'duration:[0 TO 30]'  # Set default filter
    pager = freesound_client.text_search(
        query = query,
        filter = filter,
        fields = ','.join(FREESOUND_STORE_METADATA_FIELDS),
        group_by_pack = 1,
        page_size = num_results
    )
    return [sound for sound in pager]

def retrieve_sound_preview(sound, directory):
    """Download the high-quality OGG sound preview of a given Freesound sound object to the given directory.
    """
    return freesound.FSRequest.retrieve(
        sound.previews.preview_hq_ogg,
        freesound_client,
        os.path.join(directory, sound.previews.preview_hq_ogg.split('/')[-1])
    )

def make_pandas_record(sound):
    """Create a dictionary with the metadata that we want to store for each sound.
    """
    record = {key: sound.as_dict()[key] for key in FREESOUND_STORE_METADATA_FIELDS}
    del record['previews']  # Don't store previews dict in record
    record['freesound_id'] = record['id']  # Rename 'id' to 'freesound_id'
    del record['id']
    record['path'] = "files/" + sound.previews.preview_hq_ogg.split("/")[-1]  # Store path of downloaded file
    return record

In [15]:
# Build our collection of sounds

# Our collection of sounds is made by appending the results of a number of different queries to freesound
# The query terms, query filters and the number of results per query are all defined here.
# Information about how to define filters can be found in the Freesound API documentation: https://freesound.org/docs/api/resources_apiv2.html#request-parameters-text-search-parameters
freesound_queries = [
    {
        'query': 'piano drone',
        'filter': None,
        'num_results': 30, 
    },
    {
        'query': 'ambient atmosphere',
        'filter': None,
        'num_results': 30, 
    },
    {
        'query': 'dreamy pads',
        'filter': None,
        'num_results': 30, 
    },
]

# Do all queries and concatenate the results in a single list of sounds
sounds = sum([query_freesound(query['query'], query['filter'], query['num_results']) for query in freesound_queries],[])

# Download the sounds and save them to FILES_DIR folder
for count, sound in enumerate(sounds):
    print('Downloading sound with id {0} [{1}/{2}]'.format(sound.id, count + 1, len(sounds)))
    retrieve_sound_preview(sound, 'files/')
    
# Make a Pandas DataFrame with the metadata of our sound collection and save it
df =  pd.DataFrame([make_pandas_record(s) for s in sounds])
df.to_csv(DATAFRAME_FILENAME)
print('Saved DataFrame with {0} entries! {1}'.format(len(df), DATAFRAME_FILENAME))

# Show the contents of our DataFrame (the metadata of our source collection)
display(df)

Saved DataFrame with 90 entries! dataframe.csv


,name,username,license,tags,freesound_id,path
0,piano_drone.wav,strangehorizon,http://creativecommons.org/publicdomain/zero/1.0/,"[ambient, piano, spooky, horror, drone, loop]",648312,files/648312_1089955-hq.ogg
1,Drone1.wav,HerbertBoland,https://creativecommons.org/licenses/by/4.0/,"[bell, chord, drone, fading, piano]",28124,files/28124_129090-hq.ogg
2,Piano drone.wav,robyn.levy323,http://creativecommons.org/publicdomain/zero/1.0/,"[heavily-edited, Piano, drone]",519226,files/519226_10964254-hq.ogg
3,Piano drone,laserlife,http://creativecommons.org/publicdomain/zero/1.0/,"[piano, drone, rock, post, ambient]",191167,files/191167_2372469-hq.ogg
4,Piano Drone,joelpless,http://creativecommons.org/publicdomain/zero/1.0/,"[atmosphere, mallet, spooky, deep, dark, drone...",215287,files/215287_1108574-hq.ogg
...,...,...,...,...,...,...
85,Fizz Chord,Raining-0n-Satrn,http://creativecommons.org/publicdomain/zero/1.0/,"[space, Grainy, fizzy, artificial, drone, drea...",652553,files/652553_12396721-hq.ogg
86,Xylosoul - 2.wav,Phr4kture,http://creativecommons.org/publicdomain/zero/1.0/,"[melodic, wide, atmosphere, expansive, liquid,...",218567,files/218567_468390-hq.ogg
87,Ambient Synth Key (C Major),vanszisounddesign,http://creativecommons.org/publicdomain/zero/1.0/,"[space, keys, ambient, C-Major, synth, dreamy,...",435815,files/435815_6262563-hq.ogg
88,Swirly Synth Chord Loop,nlux,https://creativecommons.org/licenses/by/4.0/,"[atmos, atmosphere, atmospheric, relaxing, swe...",620302,files/620302_9047165-hq.ogg
